# INF 8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2018 - TP1 - Méthodes de recherche 
### Membres de l'équipe
    - Amine BELLAHSEN 1965554
    - Sanae LOTFI 1968682
    - Théo Moins 1971821



## LE VÉLO À MONTRÉAL
Chaque année, Montréal accueille à peu près 10 millions de touristes. Soucieuse de la qualité de leur séjour, Tourisme Montréal a entamé un projet de développement d’une nouvelle application mobile afin d’assister les touristes lors de leurs déplacements dans la ville. Cette application a pour but d’aider l’utilisateur à planifier sa visite des importantes attractions de la ville, de la façon la plus efficace possible (ie, sur la durée la plus courte). Étant donné qu’il a été observé que le moyen de transport privilégié des touristes pour explorer Montréal est le vélo, cette application a pour but de générer des circuits cyclables de durée minimale. Plus précisément, étant donné une liste d’attractions munie de points de départ et d’arrivée, la tâche est de proposer, à chaque fois, un chemin qui passe par toutes les attractions indiquées une seule fois, qui débute au point de départ et qui s’achève au point d’arrivée et dont la durée de trajet est minimale.

<img src="images/montreal.png" alt="" width="800"/>

Le travail demandé dans ce TP est de développer l’algorithme interne de l’application. Nous explorerons trois mécanismes de résolution différents :
1. Définition et exploration naïve d’un arbre de recherche
2. Exploration plus efficace en utilisant l’algorithme A*
3. Optimisation locale en utilisant une métaheuristique de recherche à voisinage variable (Variable Neighborhood Search, VNS)

## PRÉSENTATION DU PROBLÈME
Une façon naturelle de représenter notre problème est d’utiliser un graphe $G=(V, A)$ dirigé et complet. Chaque sommet dans $V$ est une attraction donnée et chaque arc dans $A$ représente une piste cyclable entre deux attractions distinctes. Chaque paire de sommets $i$ et $j$ est reliée par une paire d’arcs $a_{ij}$ et $a_{ji}$ dont les poids respectifs $w(a_{ij})$ et $w(a_{ji})$ ne sont pas nécessairement égaux. Concrètement, ces poids représentent la durée du trajet d’un sommet à l’autre (ainsi, $w$ est telle que $w : A \to\mathbb R^+$).

La liste des attractions à visiter est indiquée comme la suite $P = (p_1, ..., p_m)$ où $p_1$ et $p_m$ sont les sommets de départ et d’arrivée, respectivement.

## 1. DÉFINITION ET EXPLORATION NAÏVE D’UN ARBRE DE RECHERCHE (5 points)
Définissons un arbre de recherche $\mathcal{T}$ où chaque nœud représente une solution partielle $S$. Soient $V(S) \subseteq V$ et $A(S) \subset A$ l’ensemble des sommets visités et l’ensemble des arêtes sélectionnées, respectivement. Ainsi, le coût d’une solution est donné par :
$$g(S) = \sum_{a \in A(S)} w(a)$$

Seule l’origine est visitée initialement. Ainsi, la racine de l’arbre de recherche contient une solution partielle vide $S_{\textrm{root}}$ telle que $V(S_{\textrm{root}})=\{p_1\}$ et $A(S_{\textrm{root}}) = \emptyset$.

<img src="images/tree1.png" alt="" width="100"/>

À la suite de cela, les nœuds subséquents dans l’arbre sont tous créés en ajoutant, à chaque solution partielle $S$, un sommet subséquent dans $P\backslash V(S)$ avec l’arc correspondant dans $A$ qui relie ce sommet à la dernière attraction visitée. Le sommet $p_m$ n’est ajouté qu’à la fin, lorsqu’il est le seul sommet non encore visité. Plus formellement, si on note le sommet à ajouter $c$ et le dernier sommet visité $c'$, alors la nouvelle solution partielle obtenue est $V(S) \gets V(S) \cup \{c\}$ et $A(S) \gets A(S) \cup \{(c’,c)\}$.

Ci-dessous est un exemple de l’arbre étendu depuis sa racine où $c'$ = $p_1$ :

<img src="images/tree2.png" alt="" width="400"/>

À la fin, les feuilles de l’arbre sont des solutions complètes :

<img src="images/tree3.png" alt="" width="600"/>

### 1.1 Code
La fonction fournie ci-dessous permet d’extraire d’un fichier un graphe qui répond aux spécifications détaillées plus haut. Cette fonction retourne une $\texttt{ndarray}$ ($\texttt{graph}$) de taille $|V|\times |V|$ où $\texttt{graph[i,j]}$ représente le temps nécessaire pour traverser la piste cyclable de $i$ vers $j$.



In [1]:
import numpy as np

def read_graph():
    return np.loadtxt("montreal", dtype='i', delimiter=',')

graph = read_graph()

Notre première tâche est de définir la classe qui représente une solution partielle. Son constructeur est donné et reçoit comme argument la liste des sommets (attractions $P$) à visiter et le graphe ($G$). Celui-ci crée la solution $S_{\textrm{root}}$ avec les attributs suivants :
- $\texttt{g}$ : le coût de la solution partielle
- $\texttt{visited}$ : représente $V(S)$, discuté plus haut. Par définition, $\mathtt{vistited[-1]}$ représente le dernier sommet ajouté, $ c $.
- $\texttt{not}\_\texttt{visited}$ : représente $P\backslash V(S)$
- $\texttt{graph}$: représente le graphe G

Ensuite, il est demandé d’implanter la méthode $\texttt{add}$ qui mets à jour la solution partielle en ajoutant une nouvelle attraction à visiter parmi la liste $\texttt{not}\_\texttt{visited}$. Cette méthode reçoit comme arguments l’index du sommet à visiter parmi $\texttt{not}\_\texttt{visited}$ ainsi que le graphe courant.

Implantez $\texttt{add}$ :

In [2]:
import copy

class Solution:
    def __init__(self, places, graph):
        """
        places: a list containing the indices of attractions to visit
        p1 = places[0]
        pm = places[-1]
        """
        self.g = 0 # current cost
        self.h = None
        self.graph = graph 
        self.visited = [places[0]] # list of already visited attractions
        self.not_visited = copy.deepcopy(places[1:]) # list of attractions not yet visited
        
    def __𝚕𝚝__(𝚜𝚎𝚕𝚏, 𝚘𝚝𝚑𝚎𝚛):
        """
        method  used to overload the comparison operator "<" for our objects Solution
        It returns return true if  f(self)<f(other)
        """
        return self.g + self.h < other.g + other.h
        
    def add(self, idx):
        """
        Adds the point in position idx of not_visited list to the solution
        """
        self.g += self.graph[self.visited[-1],self.not_visited[idx]]
        self.visited.append(self.not_visited[idx])
        del self.not_visited[idx]
        
    def swap(self,i,j):
        """
        Swaps two visited nodes and updates the cost of the solution
        """
        self.visited[i],self.visited[j] = self.visited[j],self.visited[i]
        cout = 0
        for k in range(len(self.visited)-1):
            cout += self.graph[self.visited[k],self.visited[k+1]]
        self.g = cout

La prochaine étape est d’implanter une stratégie de parcours de l’arbre de recherche. Une première méthode simple est naïve est de mettre en œuvre une recherche en largeur ([Breadth-first search](https://moodle.polymtl.ca/pluginfile.php/444662/mod_resource/content/1/recherche_en_largeur.mp4), BFS).

Implantez $\texttt{bfs}$ qui mets en œuvre cette recherche. Elle prend en arguments le graphe courant ainsi que la liste des attractions à visiter $P$ et elle retourne la meilleure solution trouvée.

In [3]:
from queue import Queue

def bfs(graph, places):
    """
    Returns the best solution which spans over all attractions indicated in 'places'
    """
    solution = Solution(places, graph)
    # Creating a queue to contain the solution
    frontiere = Queue()
    # Adding the first node to the queue
    frontiere.put(solution)
    # Setting the best solution to the first node 
    best_solution = Solution(places, graph)
    # Setting its cost to a big number
    best_solution.g = float("inf")
    nbr_explored_nodes = 0
    # Starting the Breadth First Search
    while not frontiere.empty():
        nbr_explored_nodes += 1
        # Taking the first solution node in the queue
        current_solution = frontiere.get()
        # If pm is the only node that we didn't visit, we have to compare the costs
        if len(current_solution.not_visited) == 1:
            # Adding pm to the visited nodes
            current_solution.add(0)
            # If the global cost is less than the starting cost, we take it
            if current_solution.g < best_solution.g:
                best_solution = current_solution
        else:
            # we expand the current node by adding possible attractions to visit next
            for k in current_solution.not_visited[:-1]:
                copy_current_solution = copy.deepcopy(current_solution)
                copy_current_solution.add(current_solution.not_visited.index(k))
                frontiere.put(copy_current_solution)
    print("The number of explored nodes is: ", nbr_explored_nodes)
    return best_solution


### 1.2 Expérimentations

On propose trois exemples d’illustration pour tester notre recherche en largeur. Le premier exemple prend en compte 7 attractions, le second 10 et le dernier 11. Vu que cette recherche énumère toutes les solutions possibles, le troisième exemple risque de prendre un temps considérable à s’achever.

Mettez en œuvre ces expériences et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [4]:
import time 

#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  326
27
--- 0.02809739112854004 seconds ---


In [5]:
#test 2 -------------- OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  109601
30
--- 7.168994426727295 seconds ---


In [6]:
#test 3 -------------- OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  986410
26
--- 50.43657827377319 seconds ---


## 2. RECHERCHE GUIDÉE À L’AIDE DE L’ALGORITHME A\* (7.5 points)
Pour notre deuxième méthode de recherche, au lieu d’énumérer toutes les solutions possibles, nous effectuons une recherche guidée à l’aide de l’algorithme A\*. Comme vu en classe, A\* est une recherche où les nœuds à explorer sont priorisés en fonction du coût courant d’une solution $g(S)$ ainsi que d’une estimation du coût restant vers la solution finale donné par une heuristique $h(S)$.

Dans le cas d’une minimisation, $h(S)$ est une borne inférieure du coût réel restant et on priorise l’exploration des nœuds dont $f(S) = g(S)+h(S)$ est le plus petit. Avec cette méthode, la première solution complète trouvée est assurément la solution optimale.

Pour une solution donnée $S$ avec un dernier sommet visité $c$, une possible fonction $h$ est telle que :

$h(S) =$ Le poids du chemin le plus court entre $c$ et $p_m$ dans le sous graphe $G_S$ contenant les sommets $P\backslash V(S) \cup \{c\}$

Remarque que ce chemin le plus court utilisé dans le calcul de l’estimation $h$ entre l’attraction courante et l’arrivée ne passera pas nécessairement pas tous les sommets restants.


Notre algorithme A\* se présente comme ceci :
1. Définir l’arbre de recherche $\mathcal{T}$ exactement comme auparavant. Le calcul de $h$ pour la solution initiale est inutile : c’est la seule solution qu’on a.
2. Sélectionner le meilleur nœud candidat pour expansion. La solution partielle $S_b$ de ce nœud candidat est telle que :

   $$ f(S_b) \leq f(S) \quad \forall S \in \mathcal{T} \qquad S_B, S \text{ pas encore sélectionnés}$$

   Si $S_b$ est une solution complète, l’algorithme s’arrête et $S_b$ est assurément la solution optimale, sinon on continue à l’étape 3.
3. Créer des solutions subséquentes qui connectent la dernière attraction visitée à chacune des attractions restantes. Attention, on ignore l’arrivée tant que celle-ci n’est pas la seule qui reste.
 - Mettez à jour les listes des sommets visités et non visités
 - Calculez $g$ et $h$ pour chaque solution
 - Insérer la nouvelle solution partielle dans l’arbre.
4. Répéter 2 et 3.


### 2.1 Code
Commençons d’abord par compléter la classe $\texttt{Solution}$ pour prendre en compte les changements nécessaires à A\* (on a besoin notamment d’un attribut supplémentaire pour l’estimation $h$).

On verra plus tard que A\* s’implante à l’aide d’une file de priorité (priority queue). Pour que celle-ci marche, il est nécessaire de surcharger (overload) l’opérateur de comparaison « < » relatif à nos objets $\texttt{Solution}$. En sachant ce qui fait qu'une solution est meilleure qu’une autre pour l'exploration, implanter la méthode $\_\_\texttt{lt}\_\_$ dans $\texttt{Solution}$. Son prototype est $\_\_\texttt{lt}\_\_\texttt{(self, other)}$.

Maintenant, nous devons implanter la fonction d’estimation $h$. Pour cela, on utilise l’[algorithme de Dijkstra](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) pour trouver le chemin le plus court entre la dernière attraction visitée $c$ et l’arrivée $p_m$. Il est possible d’adapter cet algorithme pour qu’il s’arrête dès que le chemin le plus court entre c et pm est trouvé.

**Prescriptions d’implantation :**
- Appliquer Dijkstra pour trouver le chemin le plus court entre $c$ et $p_m$
- Retourner le poids de ce chemin

In [7]:
def fastest_path_estimation(sol):
    """
    Returns the time spent on the fastest path between 
    the current vertex c and the ending vertex pm
    """
    # retrieving the start node c
    c = sol.visited[-1]
    # retrieving the final node pm
    pm = sol.not_visited[-1]
    
    # the dictionary "nodes" does contain the minimal distance - at a given iteration - of each node to c
    nodes = {x:float("inf") for x in sol.not_visited}
    # the minimal distance of c to itself is zero
    nodes[c] = 0
    # nodes_out contains the nodes that are already explored by the djikstra method
    nodes_out = []
    
    # min_dis_node represents the node that has the least distance to the node c
    # and that is not in nodes_out
    min_dis_node = c
    
    while min_dis_node != pm:
        for (node,value) in nodes.items():
            if node not in nodes_out:
                # update the distance of node to c
                nodes[node] = min(value, nodes[min_dis_node]+graph[min_dis_node,node])
        nodes_out.append(min_dis_node)
        if len(nodes) != len(nodes_out):
            # choose the node with the minimal distance to c that is not in nodes_out
            min_dis_node = min({k : nodes[k] for k in set(nodes) - set(nodes_out) }, key=nodes.get)
            
    return nodes[pm]

Finalement, il est temps d’implanter A\*. On aura besoin d’une file de priorité qui retournera toujours le meilleur nœud candidat de $\mathcal{T}$ pour l’étendre (l’opérateur surchargé de comparaison assure cela).

**Prescriptions d’implantation (cf. détail des étapes de l’algorithme plus haut) :**
- Tant que les solutions extraites de la file de priorité ne sont pas complètes :
  *	Sélectionner et étendre le nœud extrait de la file comme détaillé plus haut
  * Calculer $g$ et $h$ pour chaque nouvelle solution partielle obtenue
  * Remettre ces solutions dans la file
- Retourner la première solution complète extraite de la file (c’est la solution optimale)

In [8]:
import heapq

def A_star(graph, places):
    """
    Performs the A* algorithm
    """

    # blank solution
    root = Solution(graph=graph, places=places)

    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, root)
    
    # retrieve the solution with the best f value in the tree
    active_sol = heapq.heappop(T)
    
    # initiating the number of explored nodes
    nbr_explored_nodes = 0
    
    # while the solution with the best f is incomplete
    while active_sol.not_visited:
        
        nbr_explored_nodes += 1
        
        # if the active solution contains places other than pm
        if len(active_sol.not_visited) > 1:
            
            for idx in range(len(active_sol.not_visited) - 1):
                # we add possible partial solutions
                copy_active_sol = copy.deepcopy(active_sol)
                # creating a new partial solution by adding a new place > g is calculated
                copy_active_sol.add(idx)
                # calculating h
                copy_active_sol.h = fastest_path_estimation(copy_active_sol)
                heapq.heappush(T, copy_active_sol)
            active_sol = heapq.heappop(T)
            
        # Otherwise, only the last attraction is left
        else:
            # we add the last attraction and return the solution
            active_sol.add(0)
    print("The number of explored nodes is: ", nbr_explored_nodes)
    return active_sol

### 2.2 Expérimentations

On ajoute un Quatrième exemple d’exécution avec 15 attractions. Là encore, mettez en œuvre ces expériences avec le nouvel algorithme A\* conçu et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [9]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  57
27
[0, 5, 13, 16, 6, 9, 4]
--- 0.03133225440979004 seconds ---


In [10]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  358
30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.11678600311279297 seconds ---


In [11]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  997
26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.3290581703186035 seconds ---


In [12]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  108397
40
[0, 3, 5, 13, 15, 18, 20, 16, 11, 12, 14, 9, 4, 2, 1]
--- 83.80060148239136 seconds ---


### 2.3 Une meilleure borne inférieure

Notre algorithme A\* est déjà beaucoup plus efficace qu’une recherche naïve. Cependant, la qualité de l’heuristique $h$ a un très grand impact sur la vitesse de A\*. Une heuristique plus serrée devrait accélérer A\* de façon significative. Notre estimation $h$ basée sur Dijkstra est très large à cause du fait qu’elle ne considère pas toutes les attractions restantes.

Une meilleure heuristique pourrait être basée sur la **Spanning Arborescence of Minimum Weight** qui s’apparente à une Minimum Spanning Tree pour graphes orientés. On propose de construire une telle Spanning Arborescence sur le reste des attractions $P\backslash V(S) \cup \{c\}$. Ici la racine est la dernière attraction visitée $c$. Une façon classique de résoudre ce problème est d’utiliser l’[algorithme de Edmonds](https://en.wikipedia.org/wiki/Edmonds%27_algorithm).

Implantez cet algorithme et refaites les expériences avec A\* en utilisant cette nouvelle heuristique :

In [13]:
from queue import Queue

def minimum_spanning_arborescence(sol):
    """
    Returns the cost to reach the vertices in the unvisited list 
    """
    copie_sol = copy.deepcopy(sol)
    nodes = copie_sol.not_visited
    root = copie_sol.visited[-1]
    
    # creating a sub-graph for the selected univisted attractions & root
    sous_graph = copie_sol.graph[[root]+copie_sol.not_visited][:,copie_sol.not_visited]
    # creating an empty list of edges
    edges = []
    # creating an empty list of weights for the edges
    weights = {}
    
    # filling edges and weights
    for i in range(sous_graph.shape[0]):
        for j in range(sous_graph.shape[1]):
            if i != j+1:
                node_i = ([root]+copie_sol.not_visited)[i]
                node_j = copie_sol.not_visited[j]
                edges.append((node_i,node_j))
                weights[(node_i,node_j)] = sous_graph[i,j]
    
    # retrieving the minimum spanning arborescence
    tree = min_arborescence(nodes, edges, weights, root)
    return sum(weights[edge] for edge in tree)

def min_arborescence(nodes, edges, weights, root):
    """
    Recursive function that returns the edges of the minimum spanning arborescence
    for a given graph
    """
    # creating an empty dictionnary that gives minimum distance to each node
    P = {}
    # creating an empty dictionnary that gives the antecedant of each node with the minimum distance
    previous_node = {}
    # creating a dict that gives for an edge coming in the cycle the edge to be kicked out inside the cycle
    edges_out = {}
    
    # __ filling P __
    for v in nodes:
        dist_to_v = float("inf")
        best_node = v
        for x in (nodes+[root]):
            if ((x,v) in edges):
                if (x == v):
                    P[(x,v)] = 0
                if (weights[(x,v)] < dist_to_v and x != v):
                    dist_to_v = weights[(x,v)]
                    best_node = x
        P[(best_node,v)] = dist_to_v
        previous_node[v] = best_node
        
    # __ detecting the existence of in P __
    is_cycle = False
    for v in nodes:
        if iscycle(nodes,P.keys(),v):
            cycle = iscycle(nodes,P.keys(),v)
            is_cycle = True
            break
    
    # __ stop condition of the recursive algorithm __
    if not is_cycle:
        return list(P.keys())
    
    # __ creating new nodes, edges and weights to be given in the recursive call __
    else:
        v_c = max(nodes+[root])+1
        new_nodes = list(set(nodes) - set(cycle)) + [v_c]
        new_edges = []
        new_weights = {}
        Real={}
        for edge in edges:
            if (edge[0] == edge[1] and edge[1] in cycle):
                    new_edges.append((v_c,v_c))
                    new_weights[(v_c,v_c)] = 0
                    Real[(v_c,v_c)] = edge
            if (edge[0] not in cycle and edge[1] in cycle):
                new_weight = weights[edge] - weights[(previous_node[edge[1]],edge[1])]
                if ((edge[0],v_c) not in new_edges):
                    new_edges.append((edge[0],v_c))
                    new_weights[(edge[0],v_c)] = new_weight
                    Real[(edge[0],v_c)] = edge
                    edges_out[(edge[0],v_c)] = (previous_node[edge[1]],edge[1])
                else:
                    if (new_weight < new_weights[(edge[0],v_c)]):
                        new_weights[(edge[0],v_c)] = new_weight
                        edges_out[(edge[0],v_c)] = (previous_node[edge[1]],edge[1])
                        Real[(edge[0],v_c)] = edge
            elif (edge[0] in cycle and edge[1] not in cycle):
                if ((v_c, edge[1]) not in new_edges):
                    new_edges.append((v_c, edge[1]))
                    new_weights[(v_c, edge[1])] = weights[edge]
                    Real[(v_c, edge[1])] = edge
                else:
                    if (weights[edge] < new_weights[(v_c, edge[1])]):
                        new_weights[(v_c, edge[1])] = weights[edge]
                        Real[(v_c, edge[1])] = edge
            elif (edge[0] not in cycle and edge[1] not in cycle):
                new_edges.append(edge)
                Real[edge] = edge
                new_weights[edge] = weights[edge]
                
        # retrieving the minimum_spanning_arborescence with a recursive call
        arbre = min_arborescence(new_nodes,new_edges,new_weights,root)
        
        # keep edge = edge coming in the cycle to keep
        keep_edge = None
        for edge in arbre:
            if edge[1] == v_c and edge[0] != v_c:
                keep_edge = edge[0]
                
        # creating the edges of the cycle
        edge_cycle = []
        for i in range(len(cycle) - 1):
            edge_cycle.append((cycle[i],cycle[i+1]))
        edge_cycle.append((cycle[-1], cycle[0]))
        
        # generating the final minimum_spanning_arborescence
        if keep_edge is not None:
            s = set([Real[e] for e in arbre] + edge_cycle) - set([edges_out[(keep_edge,v_c)]])
        else:
            s = set([Real[e] for e in arbre] + edge_cycle)
        
        return list(s)
            

def neighbors_fct(edges, node):
    """
    Getting the list of next nodes
    """
    neighbors = []
    for edge in edges:
        if edge[0] == node and edge[1] != node:
            neighbors.append(edge[1])
    return neighbors


def iscycle(nodes,edges,node):
    file = Queue()
    # we add the initial path we have
    file.put([node])
    while not file.empty():
        # we retrieve a path from the queue
        current_path = file.get()
        last_node = current_path[-1]
        neighbors = neighbors_fct(edges,last_node)
        # we check the existence of the cycle as long as neighbors is not empty
        if neighbors:
            for new_node in neighbors:
                if new_node not in current_path:
                    copy_current_path = copy.deepcopy(current_path)
                    copy_current_path.append(new_node)
                    file.put(copy_current_path)
                else:
                    cycle = current_path[current_path.index(new_node):]
                    return cycle
    return []

In [14]:
import heapq

def A_star_edmonds(graph, places):
    """
    Performs the A* algorithm
    """

    # blank solution
    root = Solution(graph=graph, places=places)

    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, root)
    
    # retrieve the solution with the best f value in the tree
    active_sol = heapq.heappop(T)
    
    # initiating the number of explored nodes
    nbr_explored_nodes = 0
    
    # while the solution with the best f is incomplete
    while active_sol.not_visited:
        
        nbr_explored_nodes += 1
        
        # if the active solution contains places other than pm
        if len(active_sol.not_visited) > 1:
            
            for idx in range(len(active_sol.not_visited) - 1):
                # we add possible partial solutions
                copy_active_sol = copy.deepcopy(active_sol)
                # creating a new partial solution by adding a new place > g is calculated
                copy_active_sol.add(idx)
                # calculating h
                copy_active_sol.h = minimum_spanning_arborescence(copy_active_sol)
                heapq.heappush(T, copy_active_sol)
            active_sol = heapq.heappop(T)
            
        # Otherwise, only the last attraction is left
        else:
            # we add the last attraction and return the solution
            active_sol.add(0)
    print("The number of explored nodes is: ", nbr_explored_nodes)
    return active_sol

In [15]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star_edmonds(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  16
27
[0, 5, 13, 16, 6, 9, 4]
--- 0.04210782051086426 seconds ---


In [16]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star_edmonds(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  18
30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.06932735443115234 seconds ---


In [17]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star_edmonds(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  38
26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.14925503730773926 seconds ---


In [18]:
# test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star_edmonds(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

The number of explored nodes is:  84
40
[0, 3, 9, 13, 15, 18, 20, 16, 11, 12, 14, 5, 4, 2, 1]
--- 1.3427271842956543 seconds ---


## 3. RECHERCHE LOCALE À VOISINAGE VARIABLE  (7.5 points)

Cette fois-ci, au lieu de construire une solution optimale depuis une solution vide, on commence d’une solution complète, non-optimale, qu’on améliore à l’aide d’une recherche locale en utilisant une recherche locale à voisinage variable ([Variable Neighborhood Search](https://en.wikipedia.org/wiki/Variable_neighborhood_search), VNS).

<img src="images/vns.png" alt="" width="800"/>

### 3.1 Code

On commence par créer une solution initiale. Celle-ci est une suite ordonnée des attractions de $p_1$ à $p_m$ dans $P$. Pour cela, on fait appel à une [recherche en profondeur (Depth-First Search, DFS)](https://moodle.polymtl.ca/pluginfile.php/445484/mod_resource/content/1/recherche_en_profondeur.mp4) qu’on arrête aussitôt qu’une solution complète est trouvée. Pour aider à diversifier la recherche, la méthode permettant de générer une solution initiale peut être randomisée de telle sorte que l'algorithme VNS puisse lancer la recherche dans différentes régions de l'espace solution. Ainsi, dans la fonction DFS, la sélection de l'enfant pour continuer la recherche doit être aléatoire.

**Prescriptions d’implantation :**
- Mettre en œuvre une recherche en profondeur
- Créer un objet $\texttt{Solution}$ relatif à cette solution
- Ajuster les attributs de cet objet avec les bonnes valeurs de coûts et d’attractions visitées
- Retourner la solution trouvée.

In [19]:
from random import shuffle, randint

def initial_sol(graph, places):
    """
    Return a completed initial solution
    """
    # shuffling intermediate attractions
    to_shuffle = places[1:-1]
    shuffle(to_shuffle)
    new_places = [places[0]] + to_shuffle + [places[-1]]
    sol = Solution(new_places, graph)
    # adding attractions to visit
    for i in range(len(sol.not_visited)):
        sol.add(0)
    return sol

Pour définir une VNS, il faut définir les $k_\textrm{max}$ voisinages de recherche locale possibles. Pour notre problème, une bonne et simple répartition des voisinages est telle qu’un voisinage $k$ correspond à la permutation de $k$-paires de sommets dans $V(S)$.

On appelle **shaking** l’étape de génération d’une solution dans le voisinage $k$. Le travail qui suit correspond à l’implantation de cette étape. $\texttt{shaking}$ admet 3 arguments que sont la solution de départ, l’indice du voisinage $k$ ainsi que le graph courant.

Attention, avant d’implanter $\texttt{shaking}$, il est nécessaire de créer une méthode $\texttt{swap}$ dans la classe $\texttt{Solution}$. Cette méthode permet de mettre en œuvre la permutation dans une solution donnée (en mettant à jour tous les attributs nécessaires pour que la solution soit cohérente).

**Prescriptions d’implantation de shaking :**
- Sélectionner au hasard deux indices $i$ et $j$ différents et tels que $i, j \in \{2,...,m-1\}$
- Faire une copie de la solution courante et faire la permutation
- Retourner la solution créée

In [20]:
import itertools

def shaking(sol,k):
    """
    Returns a solution on the k-th neighrboohood of sol
    """
    sol_copy = copy.deepcopy(sol)
    m = len(sol_copy.visited)
    # retrieving possible pairs
    pairs = list(itertools.combinations(list(range(1,m-1)), 2))
    # shuffling pairs to add randomness
    shuffle(pairs)
    if len(pairs)>= k:
        # Swapping the elements of every pair
        for i in range(k):
            sol_copy.swap(pairs[i][0], pairs[i][1])
        return sol_copy
    else:
        print("Unable to find a neibghor for k = %s !" % k)
        return None

Une dernière étape essentielle dans une VNS est l’application d’un algorithme de recherche locale à la solution issue du shaking. Pour cela, on propose la recherche locale 2-opt. Celle-ci intervertit deux arcs dans la solution, à la recherche d’une qui est meilleure.

Pour un sommet $ i $, soit $ i '$ le successeur immédiat de $ i $ dans la séquence $ V (S) $. L'algorithme 2-opt fonctionne comme suit: pour chaque paire de sommets non consécutifs $ i, j $, vérifiez si en échangeant la position des sommets $ i '$ et $ j $ entraîne une amélioration du coût de la solution. Si oui, effectuez cet échange. Ce processus se répète jusqu'à ce qu'il n'y ait plus d'échanges rentables. On réalise cette opération pour toutes les paires d’arcs éligibles à la recherche du plus petit coût.

<img src="images/2opt.png" alt="" width="800"/>

<img src="images/2opt2.png" alt="" width="800"/>


Implantez $\texttt{local}\_\texttt{search}\_\texttt{2opt}$. 

**Prescriptions d’implantation :**
- Considérer chaque paire d’indices $i = \{2,..,m-3\}$ and $j = \{i+2, m-1\}$
- Si l’échange donne un plus bas coût, on le réalise
- Répéter jusqu’à optimum local.

In [21]:
def swap_edges(sol,i,j):
    """ 
    Returns the solution where the edges are swapped 
    """
    sol_copy = copy.deepcopy(sol)
    sol_copy.visited[i],sol_copy.visited[j] = sol_copy.visited[j],sol_copy.visited[i]
    to_reverse = sol_copy.visited[min(i,j):max(i,j)+1]
    sol_copy.not_visited[min(i,j):max(i,j)+1] = sol_copy.not_visited[min(i,j):max(i,j)+1][::-1]
    cout = 0
    for k in range(len(sol_copy.visited)-1):
        cout += graph[sol_copy.visited[k],sol_copy.visited[k+1]]
    sol_copy.g = cout
    return sol_copy

def local_search_2opt(sol):
    """
    Apply 2-opt local search over sol
    """
    active_sol = copy.deepcopy(sol)
    m = len(active_sol.visited)
    pairs = list(itertools.combinations(list(range(1,m-1)), 2))
    shuffle(pairs)
    idx = 0
    # Continue exploration from the first better solution
    while idx < len(pairs):
        i = min(pairs[idx][0],pairs[idx][1])
        j = max(pairs[idx][0],pairs[idx][1])
        if (1 < abs(i - j)):
            new_sol = swap_edges(active_sol, i, j)
            if new_sol.g < active_sol.g:
                active_sol = new_sol
                shuffle(pairs)
                idx = 0
            else:
                idx += 1
        else:
            idx += 1

    return active_sol

Finalement, il est temps d'implanter notre VNS. La méthode $\texttt{vns}$ reçoit une solution complète, le graphe courant, le nombre maximal de voisinages et un temps de calcul limite. Celle-ci retourne la solution optimale trouvée

**Prescriptions d’implantation :**
- À chaque itération, la VNS génère une solution dans le k-ème voisinage (shaking) à partir de la meilleure solution courante et applique une recherche locale 2-opt dessus
- Si la nouvelle solution trouvée a un meilleur coût, mettre à jour la meilleure solution courante
- Répéter le processus jusqu'à $\texttt{t}\_\texttt{max}$

In [22]:
def vns(sol, k_max, t_max):
    """
    Performs the VNS algorithm
    """
    # setting the start time
    start_time = time.time()
    # starting from the 1st neighborhood
    k = 1
    # performing a 1st local search
    active_sol = local_search_2opt(sol)
    while (time.time() - start_time) < t_max:
        # moving to the Kth neighborhood
        new_sol = shaking(active_sol,k)
        # performing a local search
        new_sol = local_search_2opt(new_sol)
        if new_sol.g < active_sol.g:
            active_sol = new_sol
            k = 1
        elif k < k_max:
            k += 1
        else:
            k = 1
        
    return active_sol

### 3.2 Experiments

Mettez en oeuvre la VNS sur les exemples d'illustration suivants et raportez les solutions obtenue:

In [23]:
# test 1  --------------  OPT. SOL. = 27
places=[0, 5, 13, 16, 6, 9, 4]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 1.0006752014160156 seconds ---


In [24]:
#test 2  --------------  OPT. SOL. = 30
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)

print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 1.0005314350128174 seconds ---


In [25]:
# test 3  --------------  OPT. SOL. = 26
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 1.0022814273834229 seconds ---


In [26]:
# test 4  --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

40
[0, 3, 13, 15, 18, 20, 16, 11, 12, 14, 9, 5, 4, 2, 1]
--- 1.0076885223388672 seconds ---


In [27]:
# test 5 : convergence analysis --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]

N = 50
resu = 0

for i in range(N):
    sol = initial_sol(graph=graph, places=places)
    vns_sol = vns(sol=sol, k_max=10, t_max=1)
    if vns_sol.g == 40:
        resu += 1
print("Proportion for t_max = 1s : ", 100*resu/N)

resu = 0

for i in range(N):
    sol = initial_sol(graph=graph, places=places)
    vns_sol = vns(sol=sol, k_max=10, t_max=2)
    if vns_sol.g == 40:
        resu += 1
print("Proportion for t_max = 2s : ", 100*resu/N)

Proportion for t_max = 1s :  70.0
Proportion for t_max = 2s :  86.0


## 4. BONUS (1 point)

Expliquez dans quelle situation chacun des algorithmes développés est plus approprié (prenez en compte l’évolutivité du problème)

# Réponse
Dans ce TP nous avons couvert trois algorithmes de recherche: Breadth-first search, A* dont on a implanté deux heuristiques et enfin l'algorithme de recherche local à voisinage variable.

Nous remarquons que la méthode BFS est une méthode assez naïve dont le but est de faire une recherche en largeur puis comparer le coût global à notre arrivée à l'état final avec les autres coût globaux et prendre le meilleur, donc elle parcourt tout notre arbre. Cette méthode est facile à implanter, et marche bien avec un petit nombre de noeuds à visiter, mais nous remarquons qu'elle met beaucoup plus de temps à trouver la solution en augmentant le nombre de noeuds à visiter. 

L'algorithme A* est un peu plus difficile à implanter, vu qu'on a aussi besoin d'estimer la distance entre les noeuds en utilisant des heuristiques. La première heuristique a été obtenu en utilisant l'algorithme de Djikstra. Nous remarquons que cette méthode est beaucoup plus rapide que la BFS, et parcourt moins de noeuds pour trouver la solution. La deuxième heuristique a été obtenu en utilisant l'algorithme d'Edmonds. Cette heuristique permet d'approximer mieux la borne supérieure dans A* sans sur-estimer. En effet, Djikstra permet d'obtenir le chemin le plus court entre la dernière attraction visitée et pm sans forcément passer par toutes les attractions non visitées. Tandis que l'algorithme d'Edmonds assure le passage par toutes ces attractions, ce qui rend son estimation meilleure (h plus grand). Pourtant, l'algorithme d'Edmonds permet de trouver seulement une arborescence et non pas un chemin entre la dernière attraction visitée et pm, ce qui assure que cette heuristique ne sur-estime pas les coûts. La recherche d'une solution optimale est donc plus rapide et le nombre de noeuds visités est drastiquement réduit. Notons aussi que ces 2 algorithmes assurent l'optimalité de la solution trouvée (pour A*, cette optimalité est assurée lorsque l'heuristique ne sur-estime pas les coûts).

On passe ensuite à une approche plus stochastique avec l'algorithme de recherche local à voisinage variable : VNS. Cet algorithme demande d'implanter une fonction qui nous permet de passer à un k-ème voisinage et une autre qui nous permet de faire une recherche locale dans un voisinage donné. En alternant entre les deux, nous augmentons nos chances de ne pas être bloqués dans un minimum local. Le VNS n'est pas difficile à implanter mais assure n'assure pas l'optimalité de la solution trouvée. Son avantage principal est qu'il nous permet de fixer sa durée d'exécution car à n'importe quel instant nous pouvons retourner une solution complète. Pourtant, il est peut être sensible à la définition du kème voisinage.